# Projet MLOps — Prédiction du prix des maisons

**Objectif :** Construire un pipeline complet de Data Science pour prédire le prix d’une maison.

On réalise les étapes suivantes :

- **Collecte des données**
- **Nettoyage**
- **Feature Engineering**
- **Modélisation**
- **Évaluation**



In [3]:
import os
import pandas as pd
import numpy as np


## 1. Collecte des données

On récupère le dataset depuis une source publique csv

In [4]:
os.makedirs("data/raw", exist_ok=True)
os.makedirs("data/processed", exist_ok=True)
os.makedirs("models", exist_ok=True)


In [24]:
url = "https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv"
df = pd.read_csv(url)

print(df.shape)
df.head()


(20640, 10)


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


On conserve une version brute, utile pour la reproductibilité du projet.

In [ ]:
df.to_csv("data/raw/housing_raw.csv", index=False)

## 2. Nettoyage des données

On va :

- vérifier les valeurs manquantes
- séparer la variable à prédire
- faire un train/test split
- sauvegarder les données nettoyées


In [26]:
import pandas as pd

df = pd.read_csv("data/raw/housing_raw.csv")
print(df.shape)
df.head()


(20640, 10)


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


### Traitement des valeurs manquantes


In [9]:
missing = df.isna().sum().sort_values(ascending=False)
missing[missing > 0]


total_bedrooms    207
dtype: int64

###  Séparation des variables explicatives et de la cible

- variable cible (**y**): **`median_house_value`**
- variables explicatives (**X**) : toutes les autres colonnes


In [27]:
X = df.drop(columns=["median_house_value"])
y = df["median_house_value"]

print(X.shape)
print(y.shape)


(20640, 9)
(20640,)


###  Séparation Train / Test

- **train** entraîne le modèle (80%)
- **test** évalue la performance sur des données jamais vues (20%)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print("Train:", X_train.shape, y_train.shape)
print("Test :", X_test.shape, y_test.shape)


Train: (16512, 9) (16512,)
Test : (4128, 9) (4128,)


###  Sauvegarde des données préparées


In [28]:
import os

os.makedirs("data/processed", exist_ok=True)

X_train.to_csv("data/processed/X_train.csv", index=False)
X_test.to_csv("data/processed/X_test.csv", index=False)
y_train.to_csv("data/processed/y_train.csv", index=False)
y_test.to_csv("data/processed/y_test.csv", index=False)


## 3.  Feature Engineering

Le dataset contient :

- des colonnes numériques (ex: median_income, housing_median_age, etc.)
- une colonne catégorielle : ocean_proximity

Les actions à mener : 

- remplacement les valeurs manquantes
- encodage de ocean_proximity
- standardisation les variables numériques

Pour éviter les erreurs, on fait tout ça avec un **Pipeline**.


In [13]:
import pandas as pd

X_train = pd.read_csv("data/processed/X_train.csv")
X_test = pd.read_csv("data/processed/X_test.csv")
y_train = pd.read_csv("data/processed/y_train.csv").squeeze()
y_test = pd.read_csv("data/processed/y_test.csv").squeeze()

print(X_train.shape, X_test.shape)
X_train.head()


(16512, 9) (4128, 9)


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596,NEAR OCEAN
1,-118.16,33.77,49.0,3382.0,787.0,1314.0,756.0,3.8125,NEAR OCEAN
2,-120.48,34.66,4.0,1897.0,331.0,915.0,336.0,4.1563,NEAR OCEAN
3,-117.11,32.69,36.0,1421.0,367.0,1418.0,355.0,1.9425,NEAR OCEAN
4,-119.80,36.78,43.0,2382.0,431.0,874.0,380.0,3.5542,INLAND


### Pipeline de transformation

L’idée c’est de préparer les données de la même façon pour le train et le test.

- **numériques** → remplissage des valeurs manquantes + standardisation
- **catégorielles** → encodage *OneHot*

Ça évite de faire des transformations “à la main” et ça rend le projet plus propre.


In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_features = X_train.drop(columns=["ocean_proximity"]).columns
cat_features = ["ocean_proximity"]

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", num_pipeline, num_features),
    ("cat", cat_pipeline, cat_features)
])


### Transformation des données

On applique le pipeline :

- `fit_transform` sur le train (apprendre + transformer)
- `transform` sur le test (juste transformer)


In [29]:
X_train_prepared = preprocessor.fit_transform(X_train)
X_test_prepared = preprocessor.transform(X_test)

print("X_train : ", X_train_prepared.shape)
print("X_test : ", X_test_prepared.shape)


X_train :  (16512, 13)
X_test :  (4128, 13)


### Sauvegarde


In [30]:
import joblib
import os

os.makedirs("models", exist_ok=True)

joblib.dump(preprocessor, "models/preprocessor.joblib")


['models/preprocessor.joblib']

## 4. Entraînement du modèle

Maintenant que les données sont prêtes, on peut entraîner un modèle.

On commence avec un Random Forest, parce qu’il ne demande pas trop de réglages au début. L’idée c’est d’avoir un premier modèle fonctionnel, puis on regardera les performances.


In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train_prepared, y_train)


✅ Model trained


### Sauvegarde du modèle

On sauvegarde le modèle entraîné dans le dossier models/ qui servira plus tard pour l'évaluation, prédictions, ...


In [31]:
import joblib

joblib.dump(model, "models/model.joblib")


['models/model.joblib']

### Vérification rapide

On fait une première prédiction sur le *test* juste pour voir si tout fonctionne bien.


In [32]:
y_pred = model.predict(X_test_prepared)

print("Prédictions :", y_pred[:5])
print("Valeurs réelles :", y_test.values[:5])


Prédictions : [ 50921.   70244.5 464703.4 258147.5 267298. ]
Valeurs réelles : [ 47700.  45800. 500001. 218600. 278000.]


## 5. Évaluation du modèle

On mesure la performance du modèle sur le jeu de test.


In [20]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE  : {mae:,.2f}")
print(f"RMSE : {rmse:,.2f}")


MAE  : 31,465.25
RMSE : 48,781.91


### Sauvegarde des résultats

In [33]:
import json
import os

os.makedirs("report", exist_ok=True)

results = {
    "model": "RandomForestRegressor",
    "mae": float(mae),
    "rmse": float(rmse)
}

with open("report/metrics.json", "w") as f:
    json.dump(results, f, indent=4)


### Conclusion rapide

Le modèle est entraîné et évalué avec succès 
On a maintenant un pipeline complet qui va :

1. récupérer les données
2. préparer les données
3. transformer les features
4. entraîner un modèle
5. mesurer la performance

La prochaine étape (pour rendre ça encore plus propre) sera de transformer ce notebook en **jobs Python** comme demandé dans la consigne.


# Jobs

On transforme le notebook en jobs.

Chaque job correspond à une étape du pipeline :

1. `01_collect.py` → collecte des données  
2. `02_clean.py` → nettoyage + split train/test  
3. `03_features.py` → transformations (encodage + scaling)  
4. `04_train.py` → entraînement du modèle  
5. `05_evaluate.py` → évaluation + sauvegarde des métriques  


In [ ]:
import os

os.makedirs("data/raw", exist_ok=True)
os.makedirs("data/processed", exist_ok=True)
os.makedirs("models", exist_ok=True)
os.makedirs("report", exist_ok=True)
os.makedirs("jobs", exist_ok=True)


✅ Dossiers créés


## Job 01 — Collecte des données


In [35]:
code = """
import os
import pandas as pd

def main():
    os.makedirs("data/raw", exist_ok=True)

    url = "https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv"
    df = pd.read_csv(url)

    print("Dataset shape:", df.shape)

    df.to_csv("data/raw/housing_raw.csv", index=False)
    print("✅ Saved: data/raw/housing_raw.csv")

if __name__ == "__main__":
    main()
"""

with open("jobs/01_collect.py", "w", encoding="utf-8") as f:
    f.write(code)


## Job 02 — Nettoyage + Train/Test split


In [37]:
code = """
import os
import pandas as pd
from sklearn.model_selection import train_test_split

def main():
    os.makedirs("data/processed", exist_ok=True)

    df = pd.read_csv("data/raw/housing_raw.csv")

    X = df.drop(columns=["median_house_value"])
    y = df["median_house_value"]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.2,
        random_state=42
    )

    X_train.to_csv("data/processed/X_train.csv", index=False)
    X_test.to_csv("data/processed/X_test.csv", index=False)
    y_train.to_csv("data/processed/y_train.csv", index=False)
    y_test.to_csv("data/processed/y_test.csv", index=False)

if __name__ == "__main__":
    main()
"""

with open("jobs/02_clean.py", "w", encoding="utf-8") as f:
    f.write(code)


## Job 03 — Feature Engineering


In [39]:
code = """
import os
import pandas as pd
import joblib

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

def main():
    os.makedirs("models", exist_ok=True)

    X_train = pd.read_csv("data/processed/X_train.csv")
    X_test = pd.read_csv("data/processed/X_test.csv")

    num_features = X_train.drop(columns=["ocean_proximity"]).columns
    cat_features = ["ocean_proximity"]

    num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

    cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocessor = ColumnTransformer([
        ("num", num_pipeline, num_features),
        ("cat", cat_pipeline, cat_features)
    ])

    # fit sur train uniquement
    preprocessor.fit(X_train)

    joblib.dump(preprocessor, "models/preprocessor.joblib")
if __name__ == "__main__":
    main()
"""

with open("jobs/03_features.py", "w", encoding="utf-8") as f:
    f.write(code)

## Job 04 — Entraînement du modèle

In [40]:
code = """
import os
import pandas as pd
import joblib

from sklearn.ensemble import RandomForestRegressor

def main():
    os.makedirs("models", exist_ok=True)

    # load data
    X_train = pd.read_csv("data/processed/X_train.csv")
    y_train = pd.read_csv("data/processed/y_train.csv").squeeze()

    # load preprocessor
    preprocessor = joblib.load("models/preprocessor.joblib")

    # transform
    X_train_prepared = preprocessor.transform(X_train)

    # train model
    model = RandomForestRegressor(
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train_prepared, y_train)

    joblib.dump(model, "models/model.joblib")

if __name__ == "__main__":
    main()
"""

with open("jobs/04_train.py", "w", encoding="utf-8") as f:
    f.write(code)


## Job 05 — Évaluation


In [41]:
code = """
import os
import json
import numpy as np
import pandas as pd
import joblib

from sklearn.metrics import mean_absolute_error, mean_squared_error

def main():
    os.makedirs("report", exist_ok=True)

    # load data
    X_test = pd.read_csv("data/processed/X_test.csv")
    y_test = pd.read_csv("data/processed/y_test.csv").squeeze()

    # load preprocessor + model
    preprocessor = joblib.load("models/preprocessor.joblib")
    model = joblib.load("models/model.joblib")

    # transform + predict
    X_test_prepared = preprocessor.transform(X_test)
    y_pred = model.predict(X_test_prepared)

    # metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(f"MAE  : {mae:,.2f}")
    print(f"RMSE : {rmse:,.2f}")

    results = {
        "model": "RandomForestRegressor",
        "mae": float(mae),
        "rmse": float(rmse)
    }

    with open("report/metrics.json", "w") as f:
        json.dump(results, f, indent=4)

if __name__ == "__main__":
    main()
"""

with open("jobs/05_evaluate.py", "w", encoding="utf-8") as f:
    f.write(code)
